# DELPHI simulations with LHC impedance
#### Computes also the eigenvectors

### 1) Import libraries

In [1]:
%pylab notebook

import sys
import commands
from string import *
import time
import numpy as np
from copy import deepcopy
import os,re
from io_lib import *
from tables_lib import select_in_table
from particle_param import *
from Impedance import *

#from DELPHI import *
from DELPHI_EVectors import *
from LHC_param import *



Populating the interactive namespace from numpy and matplotlib


### 1bis) Results directory

In [2]:
# machine parameters
machine2save='LHC'; 
beam='1'

# CHANGE DIRECTORY HERE
# subdirectory (inside DELPHI_results/[machine2save]) where to put the results
#ResultDir='/afs/cern.ch/user/d/damorim/work/DELPHI/Notebooks/Eigenvectors/DELPHI_output/';
ResultDir='/afs/cern.ch/work/d/damorim/work/DELPHI_results/LHC/RunII/';

os.system("mkdir -p "+ResultDir);

### 2) Parameters

In [3]:
# beam parameters
e,m0,c,E0=proton_param();

#E=3e12
#V=12e6 #RF voltage



# flags for plotting and DELPHI
flagdamperimp=0; # 1 to use frequency dependent damper gain (provided in Zd,fd)
strnorm=[''];
flagnorm=0; # 1 if damper matrix normalized at current chromaticity (instead of at zero chroma)
flagplot=True; # to write impedance files by elements
nevery=1; # downsampling of the impedance (take less points than in the full model)
wake_calc=False; # True -> compute wake as well (otherwise only imp.)


# number of eigenvalues ans convergence criterion for DELPHI
kmax=1; # number of converged eigenvalues (kmax most unstable ones are converged)
kmaxplot=300; # number of kept and plotted eigenvalues (in TMCI plot)
crit=5.0e-3
abseps=1.0e-5


# scan definition

scenarioscan=np.array(['LHC_inj_450GeV_B1_2016'])
#scenarioscan=np.array(['LHC_ft_6.5TeV_B1_2016'])

print scenarioscan


model=['Nominal LHC'  for ii in scenarioscan]; # case in LHC_param
Escan=np.array([450.0e9 for ii in scenarioscan]); # Energy at injection
#Escan=np.array([6500.0e9 for ii in scenarioscan]); # Energy at flat top
subscan=np.arange(0,len(Escan))
print subscan


# setting the scans
planes=['x'];

Qpscan=np.arange(-10,41,1);
#Qpscan=np.array([-0.01,0,0.01])

dampscan=np.array([0.0]); # damper gain scan

#Nbscan=np.arange(0,15.5e11,5e10);
Nbscan=np.array([1.2e11])

Mscan=np.array([1]); # scan on number of bunches
imp_fact=1. #impedance factor


# initialize impedance model and tune shifts
tuneshiftQp=np.zeros((len(subscan),2,len(Mscan),len(Qpscan),len(dampscan),len(Nbscan),1,1,kmaxplot),dtype=complex);
tuneshiftm0Qp=np.zeros((len(subscan),2,len(Mscan),len(Qpscan),len(dampscan),len(Nbscan),1,1),dtype=complex);
eigenvectors=np.empty((len(subscan),2,len(Mscan),len(Qpscan),len(dampscan),len(Nbscan),1,1),dtype=np.object_);


imp_mod_list=[]; # complete list of impedance scenarios
wake_mod_list=[];# complete list of wake scenarios

['LHC_inj_450GeV_B1_2016']
[0]


In [4]:
for iscenario,scenario in enumerate(scenarioscan[subscan]):

    root_result=ResultDir+scenarioscan[subscan[iscenario]]+'/'
    os.system("mkdir -p "+root_result);

In [5]:
if machine2save=='LHC':
    machine_str,E,gamma,sigmaz,taub,R,Qx,Qxfrac,Qy,Qyfrac,Qs,eta,f0,omega0,omegas,dphase,Estr,V,h, M,en=LHC_param(E0,E=Escan[subscan[iscenario]],scenario=model[subscan[iscenario]])    #machine=LHC(E0,E=Escan[subscan[iscenario]],scenario=model[subscan[iscenario]])

    g,a,b=longdistribution_decomp(taub,typelong="Gaussian");
    avbetax=R/Qx;avbetay=R/Qy; # average beta functions used
    print "scenario: ",scenario

scenario:  LHC_inj_450GeV_B1_2016


### 3) Retrieving the impedance

In [17]:
print 'Loading from impedance database...'+scenario
imp_mod=[]; wake_mod=[];

imp_files_directory=ResultDir+scenario

suffix='_Allthemachine_'+Estr+'_B'+beam+'_'+scenario+'.dat';

freq_mod,Z_mod=readZ(imp_files_directory+"/Zxdip"+suffix);
Z_mod*=imp_fact

imp_mod.append(impedance_wake(a=1,b=0,c=0,d=0,plane='x',var=freq_mod,func=Z_mod));

#freq_mod,Z_mod=readZ(root_result+"Zydip"+suffix);
#Z_mod*=imp_fact


#imp_mod.append(impedance_wake(a=0,b=1,c=0,d=0,plane='y',var=freq_mod,func=Z_mod));

imp_mod_list.append(imp_mod);
wake_mod_list.append(wake_mod);

Loading from impedance database...LHC_inj_450GeV_B1_2016


### 3bis) Impedance Plot

### 4) DELPHI calculations

In [23]:
# CHANGE IF NEEDED
# Result folder
root_result=ResultDir+scenario+'/ResultsFolder_EVectors';
os.system("mkdir -p "+root_result);

print root_result

/afs/cern.ch/work/d/damorim/work/DELPHI_results/LHC/RunII/LHC_inj_450GeV_B1_2016/ResultsFolder_EVectors


In [ ]:
# Retrieve DELPHI Results
for iscenario,scenario in enumerate(scenarioscan[subscan]):

    print 'DELPHI computation for '+scenario

    # DELPHI run
    tuneshiftQp[iscenario,:,:,:,:,:,:,:,:],tuneshiftm0Qp[iscenario,:,:,:,:,:,:,:],eigenvectors[iscenario,:,:,:,:,:,:,:]=DELPHI_wrapper(imp_mod_list[iscenario],Mscan,Qpscan,dampscan,Nbscan,[omegas],[dphase],omega0,Qx,Qy,gamma,eta,a,b,taub,g,planes,nevery=nevery,particle='proton',flagnorm=0,flagdamperimp=0,d=None,freqd=None,kmax=kmax,kmaxplot=kmaxplot,crit=crit,abseps=abseps,flagm0=True,flageigenvect=True,lxplusbatch='retrieve',comment=machine_str+scenario+'_'+float_to_str(round(E/1e9))+'GeV_Z'+float_to_str(imp_fact),queue='2nw',dire=root_result+'/',flagQpscan_outside=True);

### 5) Write DELPHI output in files

##### Save tuneshifts in text files

In [ ]:
''' Output file: data vs Qp '''

for iplane,plane in enumerate(planes):
    for iM,M in enumerate(Mscan):
        for idamp,damp in enumerate(dampscan):
            for Nb in Nbscan:
                strpart=['Re','Im'];
                for ir,r in enumerate(['real','imag']):
                    for iscenario,scenario in enumerate(scenarioscan[subscan]):

                        # output files name for data vs Qp
                        Estr=float_to_str(round(Escan[subscan[iscenario]]/1e9))+'GeV';
                        fileoutdataQp=root_result+'/data_vs_Qp_'+machine_str+'_'+Estr+scenario+'_'+str(M)+'b_d'+float_to_str(damp)+'_Nb'+float_to_str(Nb/1.e11)+'e11_'+str(kmax)+'converged'+strnorm[flagnorm]+'_'+plane;
                        fileoutdataQpm0=root_result+'/data_vs_Qp_m0_'+machine_str+'_'+Estr+scenario+'_'+str(M)+'b_d'+float_to_str(damp)+'_Nb'+float_to_str(Nb/1.e11)+'e11_'+str(kmax)+'converged'+strnorm[flagnorm]+'_'+plane;
                        fileoutdata_all=root_result+'/data_vs_Qp_all_'+machine_str+'_'+Estr+scenario+'_'+str(M)+'b_d'+float_to_str(damp)+'_Nb'+float_to_str(Nb/1.e11)+'e11_'+str(kmax)+'converged'+strnorm[flagnorm]+'_'+plane;
                        
                        ts=getattr(tuneshiftQp[iscenario,iplane,iM,:,idamp,np.where(Nbscan==Nb),0,0,0],r);
                        data=np.hstack((Qpscan.reshape((-1,1)),ts.reshape((-1,1))));
                        write_ncol_file(fileoutdataQp+'_'+r+'.dat',data,header="Qp\t"+strpart[ir]+"_tuneshift")

                        tsm0=getattr(tuneshiftm0Qp[iscenario,iplane,iM,:,idamp,np.where(Nbscan==Nb),0,0],r);
                        data=np.hstack((Qpscan.reshape((-1,1)),ts.reshape((-1,1))));
                        write_ncol_file(fileoutdataQpm0+'_'+r+'.dat',data,header="Qp\t"+strpart[ir]+"_tuneshiftm0")

                        all_unstable_modes=getattr(tuneshiftQp[iscenario,iplane,iM,:,idamp,np.where(Nbscan==Nb),0,0,:],r);
                        data=np.hstack((Qpscan.reshape((-1,1)),all_unstable_modes.reshape((-1,kmaxplot))));
                        write_ncol_file(fileoutdata_all+'_'+r+'.dat',data,header="Qp\t"+strpart[ir]+"_tuneshift")



##### Save eigenvectors in numpy .npy file

In [ ]:
for iscenario,scenario in enumerate(scenarioscan[subscan]):
    np.save(root_result+'/Eigenvectors.npy',eigenvectors[iscenario,:,:,:,:,:,:,:])